In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/michael-ivrit-dataset/test_set_from_transcript.csv
/kaggle/input/michael-ivrit-dataset/train_data.csv
/kaggle/input/michael-ivrit-dataset/test_data.csv


In [2]:
!pip install transformers --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3


In [3]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.7 MB/s eta 0:00:00


In [4]:
!pip install datasets --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.18.0
    Uninstalling datasets-2.18.0:
      Successfully uninstalled datasets-2.18.0


In [5]:
import pandas as pd
from datasets import load_dataset, load_metric, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import get_peft_config, LoraConfig, TaskType, get_peft_model

# Create DataFrames for training and evaluation data
"""
train_texts = [
    "This is a positive example.",
    "This is another positive example.",
    "This is a negative example.",
    # Add more training examples here
]

train_labels = [1, 1, 0]

eval_texts = [
    "This is a positive evaluation example.",
    "This is a negative evaluation example.",
    # Add more evaluation examples here
]

eval_labels = [1, 0]

train_data = pd.DataFrame({"text": train_texts, "label": train_labels})
eval_data = pd.DataFrame({"text": eval_texts, "label": eval_labels})
"""

2024-05-16 14:45:55.592785: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 14:45:55.592885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 14:45:55.724080: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


'\ntrain_texts = [\n    "This is a positive example.",\n    "This is another positive example.",\n    "This is a negative example.",\n    # Add more training examples here\n]\n\ntrain_labels = [1, 1, 0]\n\neval_texts = [\n    "This is a positive evaluation example.",\n    "This is a negative evaluation example.",\n    # Add more evaluation examples here\n]\n\neval_labels = [1, 0]\n\ntrain_data = pd.DataFrame({"text": train_texts, "label": train_labels})\neval_data = pd.DataFrame({"text": eval_texts, "label": eval_labels})\n'

In [6]:
import string 
import re
translator = str.maketrans('', '', string.punctuation) 

In [7]:
def calc_target(row):
    orig_text = row['orig_text'].translate(translator).split()
    text = row['text'].translate(translator).split()
    return 1 if orig_text == text  else 0

In [8]:
train_data_org = pd.read_csv('/kaggle/input/michael-ivrit-dataset/train_data.csv', index_col=None).dropna()#.head(100)
eval_data_org = pd.read_csv('/kaggle/input/michael-ivrit-dataset/test_data.csv', index_col=None).dropna()#.head(100)
train_data_org['label'] = train_data_org.apply(calc_target, axis=1)
eval_data_org['label'] = eval_data_org.apply(calc_target, axis=1)

train_data = train_data_org[['orig_text', 'label']].rename(columns={'orig_text': 'text'})
eval_data = eval_data_org[['orig_text', 'label']].rename(columns={'orig_text': 'text'})
# Create a dataset object
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)

In [9]:
# Load the pre-trained tokenizer and model
model_name = "xlm-roberta-large"
# model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

# Tokenize the texts
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/55234 [00:00<?, ? examples/s]

Map:   0%|          | 0/8425 [00:00<?, ? examples/s]

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

# Configure the LoRA parameters
# peft_config = LoraConfig(model, TaskType.SEQ_CLS, LoraConfig(r=16, lora_alpha=32))
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, inference_mode=False, r=32, lora_alpha=16, lora_dropout=0.1, bias="all"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 4,468,738 || all params: 564,089,860 || trainable%: 0.7922


In [11]:
# Initialize the Trainer
training_args = TrainingArguments(
    output_dir="./results",
#     learning_rate=1e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=25,
#     weight_decay=0.01,
#     logging_dir="./logs",
    logging_steps=10,
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    num_train_epochs=25,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to='tensorboard'
)

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Evaluate the model
eval_result = trainer.evaluate()

print(f"Evaluation result: {eval_result}")

/tmp/ipykernel_25/4240601271.py:23: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,0.658400,0.648317,0.635371
1,0.625700,0.593887,0.691395
2,0.587800,0.581347,0.705163
4,0.596400,0.579745,0.709080
5,0.618900,0.575659,0.716083
6,0.614800,0.579009,0.716558
8,0.604300,0.576938,0.716320
9,0.607800,0.576466,0.717270
10,0.577800,0.583983,0.712404
12,0.564400,0.579555,0.712404


Evaluation result: {'eval_loss': 0.5732488632202148, 'eval_accuracy': 0.7207121661721069, 'eval_runtime': 72.0567, 'eval_samples_per_second': 116.922, 'eval_steps_per_second': 7.314, 'epoch': 24.99275991891109}


In [12]:
model.save_pretrained("output_dir")

In [13]:
import torch


In [14]:
outputs = trainer.predict(tokenized_train_dataset)
preds = torch.softmax(torch.tensor(outputs.predictions), dim=1)[:, 1].tolist()

# Add the predictions to the DataFrame
train_data['probabilities'] = preds

# Print the DataFrame
train_data.to_csv('train_preds.csv')

In [15]:
outputs = trainer.predict(tokenized_eval_dataset)
preds = torch.softmax(torch.tensor(outputs.predictions), dim=1)[:, 1].tolist()

# Add the predictions to the DataFrame
eval_data['probabilities'] = preds

# Print the DataFrame
eval_data.to_csv('test_preds.csv')